In [ ]:
def preprocess_players(data):
    player_info = data['player']
    stats = data['statistics'][0] if data['statistics'] else {}
    
    player_dict = {
        'player_id': player_info['id'],
        'player_name': player_info['name'],
        'firstname': player_info['firstname'],
        'lastname': player_info['lastname'],
        'photo': player_info['photo'],
        'age': player_info['age'],
        'nationality': player_info['nationality'],
        'height': player_info['height'],
        'weight': player_info['weight'],
        'injured': player_info['injured'],
        'team_id': stats.get('team', {}).get('id', None),
        'team_name': stats.get('team', {}).get('name', None),
        'league_id': stats.get('league', {}).get('id', None),
        'season': stats.get('league', {}).get('season', None),
        'appearences': stats.get('games', {}).get('appearences', None),
        'lineups': stats.get('games', {}).get('lineups', None),
        'minutes': stats.get('games', {}).get('minutes', None),
        'position': stats.get('games', {}).get('position', None),
        'rating': stats.get('games', {}).get('rating', None),
        'captain': stats.get('games', {}).get('captain', None),
        'substitutes_in': stats.get('substitutes', {}).get('in', None),
        'substitutes_out': stats.get('substitutes', {}).get('out', None),
        'substitutes_bench': stats.get('substitutes', {}).get('bench', None),
        'shots_total': stats.get('shots', {}).get('total', None),
        'shots_on': stats.get('shots', {}).get('on', None),
        'goals_total': stats.get('goals', {}).get('total', None),
        'goals_conceded': stats.get('goals', {}).get('conceded', None),
        'goals_assists': stats.get('goals', {}).get('assists', None),
        'goals_saves': stats.get('goals', {}).get('saves', None),
        'passes_total': stats.get('passes', {}).get('total', None),
        'passes_key': stats.get('passes', {}).get('key', None),
        'passes_accuracy': stats.get('passes', {}).get('accuracy', None),
        'tackles_total': stats.get('tackles', {}).get('total', None),
        'tackles_blocks': stats.get('tackles', {}).get('blocks', None),
        'tackles_interceptions': stats.get('tackles', {}).get('interceptions', None),
        'duels_total': stats.get('duels', {}).get('total', None),
        'duels_won': stats.get('duels', {}).get('won', None),
        'dribbles_attempts': stats.get('dribbles', {}).get('attempts', None),
        'dribbles_success': stats.get('dribbles', {}).get('success', None),
        'dribbles_past': stats.get('dribbles', {}).get('past', None),
        'fouls_drawn': stats.get('fouls', {}).get('drawn', None),
        'fouls_committed': stats.get('fouls', {}).get('committed', None),
        'cards_yellow': stats.get('cards', {}).get('yellow', None),
        'cards_yellowred': stats.get('cards', {}).get('yellowred', None),
        'cards_red': stats.get('cards', {}).get('red', None),
        'penalty_won': stats.get('penalty', {}).get('won', None),
        'penalty_committed': stats.get('penalty', {}).get('commited', None),
        'penalty_scored': stats.get('penalty', {}).get('scored', None),
        'penalty_missed': stats.get('penalty', {}).get('missed', None),
        'penalty_saved': stats.get('penalty', {}).get('saved', None)
    }
    
    return player_dict

In [1]:
def get_data(endpoint, params):
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [71]:
import requests


endpoint = 'teams'
params= {'league': 39,
         'season': 2023}
df, remain = get_data(endpoint, params)

requests before reaching limit 80


In [75]:
df

{'get': 'teams',
 'parameters': {'league': '39', 'season': '2023'},
 'errors': [],
 'results': 20,
 'paging': {'current': 1, 'total': 1},
 'response': [{'team': {'id': 33,
    'name': 'Manchester United',
    'code': 'MUN',
    'country': 'England',
    'founded': 1878,
    'national': False,
    'logo': 'https://media.api-sports.io/football/teams/33.png'},
   'venue': {'id': 556,
    'name': 'Old Trafford',
    'address': 'Sir Matt Busby Way',
    'city': 'Manchester',
    'capacity': 76212,
    'surface': 'grass',
    'image': 'https://media.api-sports.io/football/venues/556.png'}},
  {'team': {'id': 34,
    'name': 'Newcastle',
    'code': 'NEW',
    'country': 'England',
    'founded': 1892,
    'national': False,
    'logo': 'https://media.api-sports.io/football/teams/34.png'},
   'venue': {'id': 562,
    'name': "St. James' Park",
    'address': 'St. James&apos; Street',
    'city': 'Newcastle upon Tyne',
    'capacity': 52758,
    'surface': 'grass',
    'image': 'https://media.